In [1]:
pip install meteostat


Note: you may need to restart the kernel to use updated packages.


In [2]:
import meteostat

https://dev.meteostat.net/python/#installation

In [48]:
#Attempting to get to weather stations as a list: 
import requests

url = 'https://meteostat.p.rapidapi.com/stations/daily'
headers = {
    'x-rapidapi-host': 'meteostat.p.rapidapi.com',
    'x-rapidapi-key': '6dcee2b33a17e72cea898af1e76d14a6'  # Replace {your_api_key} with your actual API key
}
params = {
    'station': '10637',
    'start': '2020-01-01',
    'end': '2020-01-01'
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    # Process the data as needed
    print(data)
else:
    print('Error:', response.status_code)


{'meta': {'generated': '2024-05-07 20:45:05'}, 'data': [{'date': '2020-01-01', 'tavg': 0.4, 'tmin': -3.6, 'tmax': 4.5, 'prcp': 0.1, 'snow': 0.0, 'wdir': 47.0, 'wspd': 10.1, 'wpgt': 22.3, 'pres': 1035.5, 'tsun': 438}]}


In [59]:
#Attempting to get to weather stations as DF 
import requests
import pandas as pd

url = 'https://meteostat.p.rapidapi.com/stations/daily'
headers = {
    'x-rapidapi-host': 'meteostat.p.rapidapi.com',
    'x-rapidapi-key': '2a4c4f60fdmshff28636b9a3f8c0p18f5f5jsn3b317a6c373b'  # Replace {your_api_key} with your actual API key
}
params = {
    'station': '10637',
    'start': '2020-01-01',
    'end': '2020-01-11'
}

# Make the API request
response = requests.get(url, headers=headers, params=params)

# Extract JSON data from the response
json_data = response.json()

# Normalize the JSON data into a DataFrame
weather_df = pd.json_normalize(json_data['data'], 
                               record_path=None, 
                               meta=['meta'])

# Print or use the DataFrame as needed
weather_df


,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2020-01-01,0.4,-3.6,4.5,0.1,0.0,47.0,10.1,22.3,1035.5,438
1,2020-01-02,-0.5,-2.4,1.4,0.0,0.0,198.0,6.1,25.9,1031.2,0
2,2020-01-03,5.8,1.3,9.3,0.5,0.0,209.0,15.5,46.8,1024.4,0
3,2020-01-04,5.4,2.4,6.8,0.2,0.0,242.0,15.8,47.5,1031.5,0
4,2020-01-05,4.8,2.7,6.3,1.1,0.0,189.0,6.8,23.4,1036.6,0
5,2020-01-06,1.1,-0.9,4.6,0.0,0.0,198.0,6.5,19.8,1030.6,0
6,2020-01-07,3.4,-0.1,8.0,0.0,0.0,198.0,9.7,22.3,1029.9,168
7,2020-01-08,4.9,2.8,7.5,10.6,0.0,197.0,11.9,27.7,1028.9,0
8,2020-01-09,9.2,7.4,11.2,8.6,0.0,191.0,11.5,29.5,1019.9,0
9,2020-01-10,8.3,4.0,11.6,2.1,0.0,222.0,15.8,44.3,1021.6,168


Follow this for API key: 
https://rapidapi.com/developer/authorization/default-application_9067818

In [ ]:
  url: 'https://meteostat.p.rapidapi.com/stations/meta',
  params: {id: '10637'},
  headers: {
    'X-RapidAPI-Key': '2a4c4f60fdmshff28636b9a3f8c0p18f5f5jsn3b317a6c373b',
    'X-RapidAPI-Host': 'meteostat.p.rapidapi.com'
  }

#Create location directory: 
location_directory = ["Sao Paulo",
          "Cape Town",
          "Havana"]

#Define root url
url = 'http://api.openweathermap.org/data/2.5/weather?'

#Define parameters to be added to url
parameters = {
    'appid': os.getenv('openweather_api_key'), 
    'units': 'metric',
    'lang': 'pt'}

#Define how the output will be displayed
weather_current_df = pd.DataFrame([])

#For each directory entry, define what should happen: 
for each_location in location_directory:
  #Pass on the parameters for each location:
  parameters["q"] = each_location #Check what this 'q' means 
  #The response is a REQUEST to fetch data, which is successful by following the url AND parameters:
  response = requests.get(url, parameters)
  #The output will be a response in javascript object notation, JSON format:
  weather_json_response = response.json()
  #Given the json format, next NORMALIZE it, so it can be outputted as a DATAFRAME: 
  weather_json_to_df = pd.json_normalize(weather_json_response, 
                                    sep="_", #naming convention, e.g. coord_lon, coord_lat
                                    record_path="weather", 
                                    meta=["id", #I can also leave out the meta completely 
                                          "name",
                                          "cod",
                                          "timezone",
                                          ["coord", "lon"], 
                                            ["coord", "lat"], 
                                            "base",
                                            ["main", "temp"],
                                            ["main", "feels_like"],
                                            ["main", "temp_min"],
                                            ["main", "temp_max"], 
                                            ["main", "pressure"], 
                                            ["main", "humidity"], 
                                            "visibility", 
                                            ["wind", "speed"], 
                                            ["wind", "deg"], 
                                            ["clouds", "all"], 
                                            "dt", 
                                            ["sys", "type"], 
                                            ["sys", "id"],
                                            ["sys", "country"],
                                            ["sys", "sunrise"],
                                            ["sys", "sunset"]], 
                                    record_prefix="weather_") #Insert this prefix in every column name
  #Concatenate the DFs: one was the normalized one from the JSON response, and the other one was the empty one created earlier
  weather_current_df = pd.concat([weather_current_df, weather_json_to_df], ignore_index=True)
weather_current_df

In [55]:
import requests

def get_nearest_station(lat, lon):
    url = 'https://meteostat.p.rapidapi.com/stations/nearby'
    headers = {
        'x-rapidapi-host': 'meteostat.p.rapidapi.com',
        'x-rapidapi-key': '2a4c4f60fdmshff28636b9a3f8c0p18f5f5jsn3b317a6c373b'  # Replace with your actual API key
    }
    params = {
        'lat': lat,
        'lon': lon
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data['data'][0]['id'] if data['data'] else None

# Example usage
latitude = 38.9445
longitude = -77.4558
station_code = get_nearest_station(latitude, longitude)
print(station_code)  # This will print the nearest weather station code

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [58]:
# Import Meteostat library
from meteostat import Stations

# Example usage
latitude = 38.9445
longitude = -77.4558

# Get nearby weather stations
stations = Stations()
stations = stations.nearby(latitude, longitude)
station = stations.fetch(1)

# Print DataFrame
print(station)

                               name country region    wmo  icao  latitude  \
id                                                                          
72403  Dulles International Airport      US     VA  72403  KIAD   38.9333   

       longitude  elevation          timezone hourly_start hourly_end  \
id                                                                      
72403     -77.45       95.0  America/New_York   1973-01-01 2024-05-07   

      daily_start  daily_end monthly_start monthly_end     distance  
id                                                                   
72403  1960-04-01 2024-12-30    1961-01-01  2022-01-01  1342.616722  


In [ ]:
IAD = Point(38.9445, -77.4558, 95)
BWI = Point(39.1774, -76.6684, 146)
PHL = Point(39.8721, -75.2431, 36)
EWR = Point(40.6895, -74.1745, 18)
JFK = Point(40.6413, -73.7781, 13)
LGA = Point(40.7769, -73.874, 21)

In [41]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily

# Set time period
start = datetime(2010, 2, 1)
end = datetime(2010, 2, 28)

# Create Point for Vancouver, BC
DCA = Point(38.8521, -77.0377, 0)

# Get daily data for 2018
data = Daily(DCA, start, end)
data = data.fetch()
data
# Plot line chart including average, minimum and maximum temperature
#data.plot(y=['tavg', 'tmin', 'tmax'])
#plt.show()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2010-02-01,-0.8,-6.7,3.9,0.0,102.0,NaN,8.3,NaN,1026.6,NaN
2010-02-02,0.5,-2.8,5.0,5.1,51.0,NaN,6.8,NaN,1022.2,NaN
2010-02-03,2.4,-1.1,5.6,1.5,127.0,323.0,9.4,NaN,1020.1,NaN
2010-02-04,2.4,-0.6,5.0,0.0,25.0,NaN,12.2,NaN,1028.9,NaN
2010-02-05,1.0,0.0,2.8,19.6,0.0,NaN,12.2,NaN,1021.6,NaN
2010-02-06,-2.4,-6.1,0.0,18.5,330.0,7.0,25.9,NaN,1004.7,NaN
2010-02-07,-4.3,-8.9,0.0,0.0,457.0,316.0,13.3,NaN,1016.2,NaN
2010-02-08,-2.6,-6.1,1.1,0.0,356.0,292.0,13.7,NaN,1020.9,NaN
2010-02-09,-1.7,-5.6,2.2,9.7,330.0,NaN,6.1,NaN,1017.2,NaN
